In [3]:
import pandas as pd
from pathlib import Path
from nltk.tokenize import sent_tokenize

In [4]:
files = list(Path('../data/Manifestos/Original').glob('*.csv'))
save_to = Path('../data/Manifestos/Processed')
save_to.mkdir(exist_ok=True)

In [5]:
party_mapping = {'51320':'Labour','51620':'Conservative'}

In [9]:
out = []

for f in files:
    party, date = f.stem.split('_')
    year = date[:4]
    if party in ['51320','51620']:
        df = pd.read_csv(f)
        text = ' '.join([str(i) for i in list(df.text)])
        sents = sent_tokenize(text)
        for s in sents:
            if len(s.split()) > 10:
                out.append((party_mapping[party],year,s))
    #with open(save_to / (f.stem + '.txt'),'w') as out_txt:
    #    out_txt.write(text)

In [11]:
df_data = pd.DataFrame(out,columns=['party','year','sent'])
df_data.head()

,party,year,sent
0,Conservative,1964,"""PROSPERITY WITH A PURPOSE"" Foreward by Sir ..."
1,Conservative,1964,Its object is to declare the principles for wh...
2,Conservative,1964,"Part of it is a record of achievement, and tha..."
3,Conservative,1964,For work well done carries conviction that our...
4,Conservative,1964,Our philosophy is to use what is good from the...


In [12]:
df_data.shape

(18848, 3)

In [14]:
len(df_data.sent.unique())

18829

In [18]:
df_sampled = df_data.sample(frac=1.0).reset_index(drop=True)
df_sampled.head()

,party,year,sent
0,Conservative,2005,The Right to Buy for council tenants extended ...
1,Conservative,1992,Industry has been generous with its sponsorshi...
2,Conservative,2001,The Conservative Party is committed to a compr...
3,Labour,2015,"However, Labour will legislate for a lock that..."
4,Conservative,1979,"Yet Labour have cut down our forces, weakened ..."


In [20]:
deduplicated = df_sampled.drop_duplicates(subset='sent')
deduplicated.shape

(18829, 3)

In [23]:
deduplicated['split'] = 'train'

cutoff = int(deduplicated.shape[0]*.8)
deduplicated.loc[cutoff:,'split'] = 'test'
deduplicated['split'].value_counts()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

train    15054
test      3775
Name: split, dtype: int64

In [24]:
save_to = Path('../data/Manifestos/Classifcation')
save_to.mkdir(exist_ok=True)
deduplicated.to_csv(save_to / 'labour_conservative_sentences.csv')